In [ ]:
!nvidia-smi

Mon Feb 20 13:56:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=926e9e7d26b79df2b9d195d723ba53149d2e2d825b9f3dc6a5e01bdd2055304f
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa

NER (Named-entity recognition 名词-实体识别) 分辨出文本中的名词和实体 (person人名, organization组织机构名, location地点名...).
POS (Part-of-speech tagging词性标注) 根据语法对token进行词性标注 (noun名词, verb动词, adjective形容词...)
Chunk (Chunking短语组块) 将同一个短语的tokens组块放在一起。

In [ ]:
task = 'ner' # 或者是ner, pos, chunk
model_checkpoint = 'distilbert-base-uncased'
batch_size = 16

## 加载数据

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
datasets = load_dataset('conll2003')

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
datasets['train'].features.items()

dict_items([('id', Value(dtype='string', id=None)), ('tokens', Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)), ('pos_tags', Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)), ('chunk_tags', Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)), ('ner_tags', Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None))])

In [ ]:
datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
label_list = datasets['train'].features[f'{task}_tags'].feature.names

In [ ]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
def show_elements(dataset, indices):
    df = pd.DataFrame(dataset[indices])
    display(HTML(df.to_html()))

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


数据集的tokens是按照word来的，而不是subword，所以tags标注也是按照word来标注的

In [ ]:
show_elements(datasets['train'], slice(0,9))
# show_elements(datasets['train'], range(10))
# show_elements(datasets['train'],[6372,10174,12411])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,0,"[EU, rejects, German, call, to, boycott, British, lamb, .]","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[11, 21, 11, 12, 21, 22, 11, 12, 0]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,1,"[Peter, Blackburn]","[22, 22]","[11, 12]","[1, 2]"
2,2,"[BRUSSELS, 1996-08-22]","[22, 11]","[11, 12]","[5, 0]"
3,3,"[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 35, 24, 35, 37, 16, 21, 15, 24, 41, 15, 16, 21, 21, 20, 37, 40, 35, 21, 7]","[11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 13, 11, 21, 22, 11, 12, 17, 11, 21, 17, 11, 12, 12, 21, 22, 22, 13, 11, 0]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,4,"[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 22, 38, 15, 22, 24, 20, 37, 21, 15, 24, 16, 15, 22, 15, 12, 16, 21, 38, 17, 7]","[11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 12, 21, 13, 11, 12, 21, 22, 11, 13, 11, 1, 13, 11, 17, 11, 12, 12, 21, 1, 0]","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]"
5,5,"["", We, do, n't, support, any, such, recommendation, because, we, do, n't, see, any, grounds, for, it, ,, "", the, Commission, 's, chief, spokesman, Nikolaus, van, der, Pas, told, a, news, briefing, .]","[0, 28, 41, 30, 37, 12, 16, 21, 15, 28, 41, 30, 37, 12, 24, 15, 28, 6, 0, 12, 22, 27, 16, 21, 22, 22, 14, 22, 38, 12, 21, 21, 7]","[0, 11, 21, 22, 22, 11, 12, 12, 17, 11, 21, 22, 22, 11, 12, 13, 11, 0, 0, 11, 12, 11, 12, 12, 12, 12, 12, 12, 21, 11, 12, 12, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0]"
6,6,"[He, said, further, scientific, study, was, required, and, if, it, was, found, that, action, was, needed, it, should, be, taken, by, the, European, Union, .]","[28, 38, 16, 16, 21, 38, 40, 10, 15, 28, 38, 40, 15, 21, 38, 40, 28, 20, 37, 40, 15, 12, 22, 22, 7]","[11, 21, 11, 12, 12, 21, 22, 0, 17, 11, 21, 22, 17, 11, 21, 22, 11, 21, 22, 22, 13, 11, 12, 12, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0]"
7,7,"[He, said, a, proposal, last, month, by, EU, Farm, Commissioner, Franz, Fischler, to, ban, sheep, brains, ,, spleens, and, spinal, cords, from, the, human, and, animal, food, chains, was, a, highly, specific, and, precautionary, move, to, protect, human, health, .]","[28, 38, 12, 21, 16, 21, 15, 22, 22, 22, 22, 22, 35, 37, 21, 24, 6, 24, 10, 16, 24, 15, 12, 21, 10, 21, 21, 24, 38, 12, 30, 16, 10, 16, 21, 35, 37, 16, 21, 7]","[11, 21, 11, 12, 11, 12, 13, 11, 12, 12, 12, 12, 21, 22, 11, 12, 0, 11, 0, 11, 12, 13, 11, 12, 12, 12, 12, 12, 21, 11, 12, 1, 2, 2, 11, 21, 22, 11, 12, 0]","[0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,8,"[Fischler, proposed, EU-wide, measures, after, reports, from, Britain, and, France, that, under, laboratory, conditions, sheep, could, contract, Bovine, Spongiform, Encephalopathy, (, BSE, ), --, mad, cow, disease, .]","[17, 40, 22, 42, 15, 24, 15, 22, 10, 22, 43, 15, 21, 24, 21, 20, 37, 22, 22, 22, 4, 22, 5, 8, 16, 21, 21, 7]","[11, 12, 12, 21, 13, 11, 13, 11, 12, 12, 11, 13, 11, 11, 12, 21, 22, 11, 12, 12, 0, 11, 0, 0, 11, 12, 12, 0]","[1, 0, 7, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0]"


In [ ]:
show_random_elements(datasets['train'])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,5989,"[Net, profit, 9.050, vs, 6.03]","[JJ, NN, CD, NNP, CD]","[B-NP, I-NP, I-NP, O, O]","[O, O, O, O, O]"
1,4881,"[RUGBY, LEAGUE, -, EUROPEAN, SUPER, LEAGUE, RESULTS, /, STANDINGS, .]","[NNP, NNP, :, JJ, NN, NNP, NNS, SYM, NNS, .]","[B-NP, I-NP, O, B-NP, I-NP, I-NP, I-NP, O, B-NP, O]","[B-MISC, I-MISC, O, B-MISC, I-MISC, I-MISC, O, O, O, O]"
2,10907,"[The, company, said, the, 90, percent, owned, Trax, et, al, Morinville, 10-23, logged, 28, feet, of, productive, Leduc, Reef, at, 5,350, feet, .]","[DT, NN, VBD, DT, CD, NN, VBD, NNP, VBD, JJ, NNP, CD, VBD, CD, NNS, IN, JJ, NNP, NNP, IN, CD, NNS, .]","[B-NP, I-NP, B-VP, B-NP, I-NP, I-NP, B-VP, B-NP, B-VP, B-NP, I-NP, I-NP, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, O]","[O, O, O, O, O, O, O, B-MISC, I-MISC, I-MISC, I-MISC, I-MISC, O, O, O, O, O, B-LOC, I-LOC, O, O, O, O]"
3,3765,"[Proleter, 2, 0, 1, 1, 1, 4, 1]","[NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O, O, O, O]"
4,7725,"[In, his, early, Senate, years, he, was, seen, as, a, "", prairie, populist, "", ,, working, on, legislation, protecting, farmers, ', prices, and, also, on, compensating, veterans, sickened, by, Agent, Orange, defoliant, spraying, in, the, Vietnam, War, .]","[IN, PRP$, JJ, NNP, NNS, PRP, VBD, VBN, IN, DT, "", NN, NN, "", ,, VBG, IN, NN, VBG, NNS, POS, NNS, CC, RB, IN, VBG, NNS, VBN, IN, NNP, NNP, NN, VBG, IN, DT, NNP, NNP, .]","[B-PP, B-NP, I-NP, I-NP, I-NP, B-NP, B-VP, I-VP, B-PP, B-NP, O, B-NP, I-NP, O, O, B-VP, B-PP, B-NP, B-VP, B-NP, B-NP, I-NP, O, B-ADVP, B-PP, B-NP, I-NP, B-VP, B-PP, B-NP, I-NP, I-NP, B-VP, B-PP, B-NP, I-NP, I-NP, O]","[O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MISC, I-MISC, O, O, O, O, B-MISC, I-MISC, O]"
5,1656,"[In, 1994, a, British, television, documentary, called, the, myth, around, Mother, Teresa, a, mixture, of, "", hyperbole, and, credulity, "", .]","[IN, CD, DT, JJ, NN, NN, VBD, DT, NN, IN, NNP, NNP, DT, NN, IN, "", NN, CC, NN, "", .]","[B-PP, B-NP, B-NP, I-NP, I-NP, I-NP, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-NP, I-NP, B-PP, O, B-NP, I-NP, I-NP, O, O]","[O, O, O, B-MISC, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O]"
6,8200,"[Portsmouth, 3, 1, 0, 2, 3, 5, 3]","[NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O, O, O, O]"
7,1983,"[CRICKET, -, ENGLAND, 326, ALL, OUT, V, PAKISTAN, IN, THIRD, TEST, .]","[NNP, :, NNP, CD, NNP, NNP, NNP, NNP, IN, NNP, NN, .]","[B-NP, O, B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, O]","[O, O, B-LOC, O, O, O, O, B-LOC, O, O, O, O]"
8,1779,"[(, Apr, 30, ), (, Jan, 11, )]","[(, NN, CD, ), (, NNP, CD, )]","[O, B-NP, B-NP, O, O, B-NP, I-NP, O]","[O, O, O, O, O, O, O, O]"
9,12471,"[6-, 3]","[CD, CD]","[B-NP, I-NP]","[O, O]"


## 预处理数据

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Tokenizer首先对输入进行tokenize，然后讲tokens转换为预模型中需要对应的token ID, 再转换为模型需要的输入格式。

使用AutoTokenizer.from_pretrained方法实例化我们的tokenizer，这样可以确保：
+ 我们得到一个与预训练模型一一对应的tokenizer。
+ 使用指定的模型checkpoint对应的tokenizer时，也下载了模型需要的词表库vocabulary，准确来说是tokens vocabulary。这个被下载的tokens vocabulary会被缓存起来，从而再次使用的时候不会重新下载

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

以下代码要求tokenizer必须是transformers.Pretrainedtokenzier类型，因为在预处理的时候需要用到fast tokenizer的一些特殊特性

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

tokenizer可以将word切分为subword，下面为word形式的句子

In [ ]:
example = datasets['train'][4]
print(example['tokens'])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


下面为切分为subword

In [ ]:
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
print(len(example['tokens']))
print(len(example[f'{task}_tags'])) 
print(tokenized_input['input_ids'])
print(len(tokenized_input['input_ids']))
print(tokens)
print(len(tokens))

31
31
[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]
39
['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']
39


### 数据对齐

标注数据通常在word级别进行标注，那么需要将标注数据进行subtokens的对齐，可以通过word_ids实现

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


word_ids将每一个subtokens位置对应了一个word的下标。有了这个list，就可以将subtokens和words还有标注的labels对齐

In [ ]:
word_ids=tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f'{task}_tags'][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input['input_ids']))
print(aligned_labels)

39 39
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]


有两种对齐label的方式：
+ 多个subtokens对齐一个word，对齐一个label
+ 多个subtokens的第一个subtoken对齐word，对齐一个label，其他subtokens直接赋予-100.
第一种方法设置label_all_tokens为True，第二种为False

In [ ]:
label_all_tokens=True

### 预处理函数

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    #return_tensors="pt"表示返回Pytorch张量。文本转换为数字之后必须再转换成张量tensors才能输入模型。
    #padding=True表示填充输入序列到最大长度，truncation=True表示过长序列被截断
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
print(tokenize_and_align_labels(datasets['train'][:5]))
# input_ids  attention_mask  labels

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

对数据集datasets里面的所有样本进行预处理，**处理的方式是使用map函数，将预处理函数prepare_train_features应用到（map)所有样本上/**

更好的是，返回的结果会自动被缓存，避免下次处理的时候重新计算（但是也要注意，如果输入有改动，可能会被缓存影响！）。datasets库函数会对输入的参数进行检测，判断是否有变化，如果没有变化就使用缓存数据，如果有变化就重新处理。但如果输入参数不变，想改变输入的时候，最好清理调这个缓存。清理的方式是使用load_from_cache_file=False参数。另外，上面使用到的batched=True这个参数是tokenizer的特点，因为这会使用多线程同时并行对输入进行处理。

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## 微调预训练模型

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

由于我们微调的任务是token分类任务，而我们加载的是预训练的语言模型，所以会提示我们加载模型的时候扔掉了一些不匹配的神经网络参数（比如：预训练语言模型的神经网络head被扔掉了，同时随机初始化了token分类的神经网络head）。

**？？？？？？**

In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch", # 每个epoch进行一次验证评估
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01)

**datacollator作用是什么？ 数据收集器，将处理好的输入喂给模型。**

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

<ipython-input-34-d0b6118e6d86>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
for token, tag, label in zip(example[f"tokens"],example[f"{task}_tags"], labels):
    print(token, tag, label)

Germany 5 B-LOC
's 0 O
representative 0 O
to 0 O
the 0 O
European 3 B-ORG
Union 4 I-ORG
's 0 O
veterinary 0 O
committee 0 O
Werner 1 B-PER
Zwingmann 2 I-PER
said 0 O
on 0 O
Wednesday 0 O
consumers 0 O
should 0 O
buy 0 O
sheepmeat 0 O
from 0 O
countries 0 O
other 0 O
than 0 O
Britain 5 B-LOC
until 0 O
the 0 O
scientific 0 O
advice 0 O
was 0 O
clearer 0 O
. 0 O


对模型预测结果进行一些后处理：

+ 选择预测分类最大概率的下标
+ 将下标转换为label
+ 忽略-100所在地方


将上面的步骤合并

In [ ]:
import numpy as np

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # 移除特殊token
    true_predictions = [
        [label_list[p] for (p,l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l!=-100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return{
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, chunk_tags, id, ner_tags, pos_tags. If tokens, chunk_tags, id, ner_tags, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634
  Number of trainable parameters = 66369801


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.015300,0.070928,0.930920,0.937689,0.934292,0.984987
2,0.016400,0.065773,0.931023,0.937689,0.934344,0.984701
3,0.007200,0.069954,0.932515,0.942947,0.937702,0.985496


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, chunk_tags, id, ner_tags, pos_tags. If tokens, chunk_tags, id, ner_tags, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Spec

TrainOutput(global_step=2634, training_loss=0.012196450361991889, metrics={'train_runtime': 260.9936, 'train_samples_per_second': 161.395, 'train_steps_per_second': 10.092, 'total_flos': 510709003547400.0, 'train_loss': 0.012196450361991889, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, chunk_tags, id, ner_tags, pos_tags. If tokens, chunk_tags, id, ner_tags, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16


{'eval_loss': 0.06995393335819244,
 'eval_precision': 0.9325146587011838,
 'eval_recall': 0.9429466383264348,
 'eval_f1': 0.9377016353320725,
 'eval_accuracy': 0.9854957345068073,
 'eval_runtime': 9.7589,
 'eval_samples_per_second': 333.028,
 'eval_steps_per_second': 20.904,
 'epoch': 3.0}

如果想得到单个类别的precision/recall/f1，可以直接将结果输入相同的评估函数

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, chunk_tags, id, ner_tags, pos_tags. If tokens, chunk_tags, id, ner_tags, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3250
  Batch size = 16


{'LOC': {'precision': 0.9544246107102164,
  'recall': 0.9598930481283422,
  'f1': 0.9571510188535517,
  'number': 2618},
 'MISC': {'precision': 0.8406725380304243,
  'recall': 0.8529650690495532,
  'f1': 0.846774193548387,
  'number': 1231},
 'ORG': {'precision': 0.8952067669172933,
  'recall': 0.9265564202334631,
  'f1': 0.9106118546845123,
  'number': 2056},
 'PER': {'precision': 0.977550346649059,
  'recall': 0.9759393539881345,
  'f1': 0.9767441860465117,
  'number': 3034},
 'overall_precision': 0.9325146587011838,
 'overall_recall': 0.9429466383264348,
 'overall_f1': 0.9377016353320725,
 'overall_accuracy': 0.9854957345068073}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
